In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split

C:\Users\BEU_RU1\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Задание 1

Вернемся к набору данных о видеоиграх.

Ответьте на следующие вопросы:

Как критики относятся к спортивным играм?

Критикам нравятся больше игры на PC или на PS4?

Критикам больше нравятся стрелялки или стратегии?

Для каждого вопроса:

сформулируйте нулевую и альтернативную гипотезы;

выберите пороговый уровень статистической значимости;

опишите полученные результаты статистического теста.


In [3]:
df = pd.read_csv('vgsales.csv')
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


1. Как критики относятся к спортивным играм?

Критики ставят играм в основом высокие оценки (выше 70)?

H0: средняя оценка игр критиками <= 70.0
H1: средняя оценка игр критиками > 70.0
Пусть alpha здесь и далее будем принимать 0.05

In [6]:
df_sport = df[df['Genre'] == 'Sports']
df_sport.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
13,Wii Fit,Wii,2007.0,Sports,Nintendo,8.92,8.03,3.60,2.15,22.70,80.0,63.0,7.7,146.0,Nintendo,E
15,Wii Fit Plus,Wii,2009.0,Sports,Nintendo,9.01,8.49,2.53,1.77,21.79,80.0,33.0,7.4,52.0,Nintendo,E
77,FIFA 16,PS4,2015.0,Sports,Electronic Arts,1.12,6.12,0.06,1.28,8.57,82.0,42.0,4.3,896.0,EA Sports,E


In [10]:
df_cs = df_sport[np.logical_not(df['Critic_Score'].isna())]['Critic_Score']

C:\Users\BEU_RU1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [11]:
alpha = 0.05
result = st.ttest_1samp(df_cs, 70.0)

print(df_cs.mean())
print(result)

if result.pvalue < alpha: 
    print('Отвергаем нулевую гипотезу, средняя оценка игр критиками выше 70')
else:
    print('Не отвергаем нулевую нулевую гипотезу, средняя оценка игр критиками не выше 70')

71.96817420435511
Ttest_1sampResult(statistic=4.900329475499578, pvalue=1.0885489197416262e-06)
Отвергаем нулевую гипотезу, средняя оценка игр критиками выше 70


2. Критикам нравятся больше игры на PC или на PS4?

Средняя оценка игр на PC и на PS4 одинакова?

H0: средняя оценка игр на PC и на PS4 одинакова.
H1: средняя оценка игр на PC и на PS4 различается.

In [14]:
df_cs_PC = df[np.logical_and(np.logical_not(df['Critic_Score'].isna()),
                                    df['Platform'] == 'PC')]['Critic_Score']

df_cs_PS4 = df[np.logical_and(np.logical_not(df['Critic_Score'].isna()),
                                    df['Platform'] == 'PS4')]['Critic_Score']

In [15]:
result = st.ttest_ind(df_cs_PC, df_cs_PS4, equal_var=False)
print(result)

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, средняя оценка игр на PC и на PS4 различается')
else:
    print('Не отвергаем нулевую гипотезу, средняя оценка игр на PC и на PS4 одинакова')

Ttest_indResult(statistic=4.3087588262138725, pvalue=2.067249157283479e-05)
Отвергаем нулевую гипотезу, средняя оценка игр на PC и на PS4 различается


3. Критикам больше нравятся стрелялки или стратегии?

Средняя оценка шутеров и стратегий одинакова?

H0: средняя оценка шутеров и стратегий одинакова.
H1: средняя оценка шутеров и стратегий различается.

In [17]:
df_cs_sh = df[np.logical_and(np.logical_not(df['Critic_Score'].isna()),
                                    df['Genre'] == 'Shooter')]['Critic_Score']

df_cs_st = df[np.logical_and(np.logical_not(df['Critic_Score'].isna()),
                                    df['Genre'] == 'Strategy')]['Critic_Score']

In [18]:
esult = st.ttest_ind(df_cs_sh, df_cs_st, equal_var=False)
print(result)

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, средняя оценка шутеров и стратегий различается')
else:
    print('Не отвергаем нулевую гипотезу, средняя оценка шутеров и стратегий одинакова')

Ttest_indResult(statistic=4.3087588262138725, pvalue=2.067249157283479e-05)
Отвергаем нулевую гипотезу, средняя оценка шутеров и стратегий различается


Задание 2

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:

Приведите весь текст к нижнему регистру;

Удалите мусорные символы;

Удалите стоп-слова;

Приведите все слова к нормальной форме;

Преобразуйте все сообщения в вектора TF-IDF;

Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;

Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;

Опишите результаты при помощи confusion_matrix;

Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [20]:
df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


1. Приведите весь текст к нижнему регистру

In [21]:
df['message'] = df['Message'].str.lower()
df.head()

,Category,Message,message
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro..."


2. Удалите мусорные символы

In [22]:
df['only_words'] = df['message'].map(lambda x: re.sub('[\W_]+',' ', x))
df.head()

,Category,Message,message,only_words
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...",nah i don t think he goes to usf he lives arou...


3. Удалите стоп-слова

In [24]:
df['only_words'] = df['only_words'].map(lambda x: x.split())
df.head()

,Category,Message,message,only_words
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,..."


In [31]:
pip install --user -U nltk

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [40]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BEU_RU1\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [41]:
from nltk.corpus import stopwords

In [42]:
stopwords_set = set(stopwords.words('english'))
df['without_sw'] = df['only_words'].map(lambda x: [word for word in x if word not in stopwords_set] )
df.head()

,Category,Message,message,only_words,without_sw
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,...","[nah, think, goes, usf, lives, around, though]"


4. Приведите все слова к нормальной форме

In [44]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BEU_RU1\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [45]:
wordnet_lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['without_sw'].map(lambda x: [wordnet_lemmatizer.lemmatize(word) for word in x] )
df.head()

,Category,Message,message,only_words,without_sw,lemmatized
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,...","[nah, think, goes, usf, lives, around, though]","[nah, think, go, usf, life, around, though]"


In [46]:
def to_str(lem):
    return ' '.join([x for x in lem])

In [47]:
df['new_message'] = df['lemmatized'].apply(to_str)
df.head()

,Category,Message,message,only_words,without_sw,lemmatized,new_message
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazy, available, bugis, n...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]",ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin...",free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, early, hor, u, c, already, say]",u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,...","[nah, think, goes, usf, lives, around, though]","[nah, think, go, usf, life, around, though]",nah think go usf life around though


5. Преобразуйте все сообщения в вектора TF-IDF

In [48]:
df = df.drop(columns=['message', 'only_words', 'without_sw', 'lemmatized'])
df.head()

,Category,Message,new_message
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though


In [49]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['new_message'])

names = tfidf.get_feature_names()
matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
matrix.head()

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


6. Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42.

In [52]:
df['class_message'] = (df['Category'] == 'spam') * 1
df.head()

,Category,Message,new_message,class_message
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,0
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though,0


In [53]:
X_train, X_test, y_train, y_test = train_test_split(matrix, df['class_message'], test_size=0.30, random_state=42)

7. Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных

In [54]:
lda = LinearDiscriminantAnalysis()

In [55]:
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [56]:
accuracy_score(y_test, lda.predict(X_test))

0.9677033492822966

Точность - 97%.

8. Опишите результаты при помощи confusion_matrix

In [57]:
confusion_matrix(y_test, lda.predict(X_test))

array([[1445,    3],
       [  51,  173]], dtype=int64)

Правильно предсказано 1445 единичек и 173 ноля. Также есть 51 + 3 ошибки (ноль вместо единицы или наоборот).

9. Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного)

In [58]:
df_wrong = pd.concat([df[df.index.isin(y_test.index)].reset_index(), 
                    pd.Series(lda.predict(X_test),
                              np.arange(len(lda.predict(X_test))),
                              name = 'predict')],
                    axis=1)
df_wrong = df_wrong[df_wrong['class_message'] != df_wrong['predict']]
df_wrong

,index,Category,Message,new_message,class_message,predict
0,8,spam,WINNER!! As a valued network customer you have...,winner valued network customer selected receiv...,1,0
1,12,spam,URGENT! You have won a 1 week FREE membership ...,urgent 1 week free membership 100 000 prize ja...,1,0
2,15,spam,"XXXMobileMovieClub: To use your credit, click ...",xxxmobilemovieclub use credit click wap link n...,1,0
4,19,spam,England v Macedonia - dont miss the goals/team...,england v macedonia dont miss goal team news t...,1,0
14,47,ham,"Fair enough, anything going on?",fair enough anything going,0,1
...,...,...,...,...,...,...
1633,5446,ham,I am back. Good journey! Let me know if you ne...,back good journey let know need receipt shall ...,0,1
1634,5450,ham,Sac needs to carry on:),sac need carry,0,1
1638,5457,ham,Arun can u transfr me d amt,arun u transfr amt,0,1
1657,5524,spam,You are awarded a SiPix Digital Camera! call 0...,awarded sipix digital camera call 09061221061 ...,1,0
